A file to check who we are capturing sensor data from...

# Setup

In [3]:
import numpy as np
import pandas as pd

In [4]:
import os
import zipfile

Need to check for:
- all 6 sensors
    - device usage
    - locations
    - keyboard
    - message
    - phone
    - speech
    
- health kit items
    - sleep
    - activity ?
    - heart rate
    - location


In [5]:
# Use your folder path 
path = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_2/app_data/RK.8D1DBFAD.DJW Thesis_20230101-20230308/'

In [6]:
files = os.listdir(os.chdir(path))

In [7]:
files

['SurveyResults_20230101-20230308.csv',
 'HealthKitV2Samples_EnvironmentalAudioExposure_Deleted_20230101-20230308.csv',
 'SurveyTasks_20230101-20230308.csv',
 'HealthKitV2Samples_StandHourInterval_20230101-20230308.csv',
 'HealthKitV2Samples_AudioExposureEvent_20230101-20230308.csv',
 'sensorkit-visits',
 'HealthKitV2Samples_Steps_20230101-20230308.csv',
 'HealthKitV2Samples_EnvironmentalAudioExposure_20230101-20230308.csv',
 'HealthKitV2Samples_RestingEnergyBurned_20230101-20230308.csv',
 'HealthKitV2Samples_ToothbrushingEvent_20230101-20230308.csv',
 'HealthKitV2Samples_AudioExposureEvent_Deleted_20230101-20230308.csv',
 'SurveySpatialSpanMemoryResults_20230101-20230308.csv',
 'HealthKitV2Samples_RestingHeartRate_Deleted_20230101-20230308.csv',
 'HealthKitV2Samples_DistanceCycling_Deleted_20230101-20230308.csv',
 'HealthKitV2Samples_DistanceWalkingRunning_Deleted_20230101-20230308.csv',
 'sensorkit-device-usage',
 'WeatherData_20230101-20230308.csv',
 'HealthKitV2Samples_HeartRateVar

In [353]:
good_files = []

# remove unwanted files
for file in files:
    if file.startswith('RK'):
        good_files += [file]

In [354]:
good_files.sort()

In [358]:
week3 = good_files[14:21]
week4 = good_files[21:28]

In [391]:
week4

['RK.8D1DBFAD.DJWThesis_20221018-20221019',
 'RK.8D1DBFAD.DJWThesis_20221019-20221020',
 'RK.8D1DBFAD.DJWThesis_20221020-20221021',
 'RK.8D1DBFAD.DJWThesis_20221021-20221022',
 'RK.8D1DBFAD.DJWThesis_20221022-20221023',
 'RK.8D1DBFAD.DJWThesis_20221023-20221024',
 'RK.8D1DBFAD.DJWThesis_20221024-20221025']

## Study Participants

In [65]:
df = pd.read_csv('/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_2/run2_subjects.csv')
df = df[['ParticipantIdentifier', 'SecondaryIdentifier', 'Email']]
df.shape

(93, 3)

## `HealthKitSamples`

In [66]:
# indicate how far back to go when looking at data
date = '2023-02-26'

df = df.assign(Location = 0)
df_temp = pd.read_csv(path + 'AppleLocationVisits_20230101-20230308.csv')
df.loc[df.ParticipantIdentifier.isin(np.unique(df_temp.loc[df_temp.InsertedDate > date].ParticipantIdentifier)), 'Location'] = 1

In [67]:
df = df.assign(HRVar = 0)
df_temp = pd.read_csv(path + 'HealthKitV2Samples_HeartRateVariability_20230101-20230308.csv')
df.loc[df.ParticipantIdentifier.isin(np.unique(df_temp.loc[df_temp.InsertedDate > date].ParticipantIdentifier)), 'HRVar'] = 1

df = df.assign(HKActiveEnergy = 0)
df_temp = pd.read_csv(path + 'HealthKitV2Samples_ActiveEnergyBurned_20230101-20230308.csv')
df.loc[df.ParticipantIdentifier.isin(np.unique(df_temp.loc[df_temp.InsertedDate > date].ParticipantIdentifier)), 'HKActiveEnergy'] = 1

/Users/djw/opt/miniconda3/envs/thesis_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [78]:
df.head(3)

,ParticipantIdentifier,SecondaryIdentifier,Email,Location,HRVar,HKActiveEnergy,SKdeviceUsage,SKkeyboardMetrics,SKmessagesUsage,SKspeechTelephony,SKvisits
0,d5f19109-8219-40e6-9fc1-6e02c3a06dcd,recu1JXHWEL5JXhGa,keyi.huang@mail.utoronto.ca,1,1,1,1,1,1,1,1
1,852c24f8-36d6-4bbd-b79f-7f6fe64d1275,recPySBG195zlOcxk,yunyi.zhang@mail.utoronto.ca,0,1,1,1,1,1,1,1
2,35d11ffc-7034-4708-a086-cd4bd47b51fd,rec2HFeoNLSvyHlcq,irene.guerrero@mail.utoronto.ca,1,1,1,1,1,1,1,1


In [70]:
df = df.assign(SKdeviceUsage = 0)

subs = os.listdir(os.chdir(path + 'sensorkit-device-usage/iPhone'))

for sub in subs:
    device = os.listdir(os.chdir(path + 'sensorkit-device-usage/iPhone/' + sub))
    files = os.listdir(os.chdir(path + 'sensorkit-device-usage/iPhone/' + sub + '/' + device[0]))
    x = [file_date for file_date in files if file_date  > date]
    if len(x) > 0:
        df.loc[df.ParticipantIdentifier == sub, 'SKdeviceUsage'] = 1

In [71]:
df = df.assign(SKkeyboardMetrics = 0)
subs = os.listdir(os.chdir(path + 'sensorkit-keyboard-metrics/iPhone'))

for sub in subs:
    device = os.listdir(os.chdir(path + 'sensorkit-keyboard-metrics/iPhone/' + sub))
    files = os.listdir(os.chdir(path + 'sensorkit-keyboard-metrics/iPhone/' + sub + '/' + device[0]))
    x = [file_date for file_date in files if file_date  > date]
    if len(x) > 0:
        df.loc[df.ParticipantIdentifier == sub, 'SKkeyboardMetrics'] = 1

In [73]:
df = df.assign(SKmessagesUsage = 0)

subs = os.listdir(os.chdir(path + 'sensorkit-messages-usage/iPhone'))

for sub in subs:
    device = os.listdir(os.chdir(path + 'sensorkit-messages-usage/iPhone/' + sub))
    files = os.listdir(os.chdir(path + 'sensorkit-messages-usage/iPhone/' + sub + '/' + device[0]))
    x = [file_date for file_date in files if file_date  > date]
    if len(x) > 0:
        df.loc[df.ParticipantIdentifier == sub, 'SKmessagesUsage'] = 1

In [75]:
df = df.assign(SKspeechTelephony = 0)

subs = os.listdir(os.chdir(path + 'sensorkit-speech-telephony/iPhone'))

for sub in subs:
    device = os.listdir(os.chdir(path + 'sensorkit-speech-telephony/iPhone/' + sub))
    files = os.listdir(os.chdir(path + 'sensorkit-speech-telephony/iPhone/' + sub + '/' + device[0]))
    x = [file_date for file_date in files if file_date  > date]
    if len(x) > 0:
        df.loc[df.ParticipantIdentifier == sub, 'SKspeechTelephony'] = 1

In [77]:
df = df.assign(SKvisits = 0)

subs = os.listdir(os.chdir(path + 'sensorkit-visits/iPhone'))

for sub in subs:
    device = os.listdir(os.chdir(path + 'sensorkit-visits/iPhone/' + sub))
    files = os.listdir(os.chdir(path + 'sensorkit-visits/iPhone/' + sub + '/' + device[0]))
    x = [file_date for file_date in files if file_date  > date]
    if len(x) > 0:
        df.loc[df.ParticipantIdentifier == sub, 'SKvisits'] = 1

In [85]:
# make email all lowercase so sorting works
df['Email'] = df['Email'].apply(str.lower)
# sort by email so can copy paste to Airtable
df = df.sort_values(by = ['Email'])

df.to_csv(path + 'adherenceCheck.csv', index=False)

In [87]:
df.sum()

ParticipantIdentifier    98f7c7df-3bbf-44bf-99be-e2995f557e91ec83dfe2-3...
SecondaryIdentifier      rec27fWK4UmKDxN8urecS93j78tHbSdGe8recraaTSlAyl...
Email                    abigale.richards@mail.utoronto.caabisha.jeyapa...
Location                                                                54
HRVar                                                                   87
HKActiveEnergy                                                          90
SKdeviceUsage                                                           83
SKkeyboardMetrics                                                       68
SKmessagesUsage                                                         91
SKspeechTelephony                                                       76
SKvisits                                                                78
dtype: object